# **Vocabulary challenge**
_Felipe Andres Castillo_

In [1]:
import pymupdf
import csv
import re
import pandas as pd
import numpy as np

## 1. Obtención de palabras

In [2]:
def standardize_word(word):
    # Se elimina cualquier caracter especial que no sea una letra
    std_word = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ]', '', word)
    # Se covierten todas las letras a minusculas
    return std_word.lower()

In [3]:
# Se abre el archivo
doc = pymupdf.open("Los miserables.pdf")

# Esta lista contendrá todas las palabras extraidas del PDF
words_list = []
# Obtención de palabras
for page in doc:
    # Por cada página del PDF, con .get_text() se obtiene una lista de listas, 
    # donde cada lista está asociada a cada palabra   
    words = page.get_text("words")  
    for word in words:
        # Cada lista (words) tiene el formato (x0, y0, x1, y1, "word", block_no, line_no, word_no) 
        # siendo los primeros 4 valores las coordenadas de la palabra, y la siguiente (quinta posición) es la palabra en sí
        # FORMATEO de cada palabra: 
        std_word = standardize_word(word[4])
        if std_word != "":
            words_list.append([std_word]) 

# Guardar CSV con palabras formateadas
with open("palabras.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(words_list)

## 2. Creación del vocabulario

In [ ]:
# Creación de la clase Word, la cual tiene dos atributos: la palabra y su frecuencia de aparición
# class Word:
#     def __init__(self, word, freq):
#         self.word = word
#         self.freq = freq
#     def __repr__(self):
#         return f"({self.word}, {self.freq})"
#     def __str__(self):
#         return f"Word: {self.word}, Frequency: {self.freq}" 

In [4]:
# Diccionario-Vocabulario -> apoyo para determinar la frecuencia de cada palabra
Vocabulary_dict = {}

with open("palabras.csv", "r", encoding="utf-8") as file:
    text = csv.reader(file)
    for word in text:
        if word[0] in Vocabulary_dict:
            Vocabulary_dict[word[0]] += 1
        else:
            Vocabulary_dict[word[0]] = 1  

In [5]:
# DataFrame-Vocabulario -> para estadistica de los valores de frecuencia obtenidos
Vocabulary_df = pd.DataFrame.from_dict(Vocabulary_dict, orient="index", columns=["frequency"]).reset_index()
Vocabulary_df.columns = ["words", "frequency"]

In [6]:
# Se guarda el Vocabulario en formato parquet (en particular, que se guarden en orden según su frecuencia)
df = Vocabulary_df.sort_values(by = "frequency", ascending=False)
df.to_parquet("Vocabulary.parquet", engine="pyarrow", index=False)

In [7]:
# Lectura desde el archivo parquet generado
Vocabulary = pd.read_parquet("Vocabulary.parquet", engine="pyarrow")
print(Vocabulary)

             words  frequency
0               de       5321
1               la       3917
2              que       3505
3                y       3122
4               el       3081
...            ...        ...
13235     seguidas          1
13236       mintió          1
13237      evadido          1
13238      insista          1
13239  desfallecer          1

[13240 rows x 2 columns]


## 3. Algunas estadísticas 

¿Cuántas palabras hay en total?

In [8]:
print(f"Hay {len(words_list)} palabras en el PDF")

Hay 109224 palabras en el PDF


¿Cuántas palabras diferentes hay en el libro?

In [9]:
print(f"Hay {len(Vocabulary)} palabras diferentes")

Hay 13240 palabras diferentes


¿Cuáles son las 10 palabras más frecuentes?

In [10]:
Vocabulary[0:10]

,words,frequency
0,de,5321
1,la,3917
2,que,3505
3,y,3122
4,el,3081
5,en,2835
6,a,2488
7,se,1632
8,un,1601
9,no,1498


¿Cuáles son las 10 palabras menos frecuentes?

In [11]:
Vocabulary[-10:]

,words,frequency
13230,bujía,1
13231,singularidad,1
13232,reparó,1
13233,valga,1
13234,holocausto,1
13235,seguidas,1
13236,mintió,1
13237,evadido,1
13238,insista,1
13239,desfallecer,1


¿Cuántas palabras solo aparecen una vez?

In [12]:
once = Vocabulary[Vocabulary.frequency == 1]
print(f"Hay {len(once)} palabras que solo aparecen una vez") 

Hay 7297 palabras que solo aparecen una vez


Una función para saber cuantas palabras hay con determinada frecuencia:

In [13]:
def f_word(vocabulary, freq):
    sub_voc = Vocabulary[Vocabulary.frequency == freq]
    print(f"Hay {len(sub_voc)} palabras con frecuencia {freq}")

In [14]:
# Ejemplo
f_word(Vocabulary, 18)

Hay 42 palabras con frecuencia 18


## 4. Eliminando algunos nexos

Los nexos, en el contexto del lenguaje y la gramática, son palabras o grupos de palabras que se utilizan para conectar oraciones, frases o partes de un texto con el fin de establecer relaciones lógicas y coherencia en la comunicación escrita o hablada. Es natural que las palabras más frecuentes sean ejemplos de nexos. Si omitimos aquellas palabras que tengan menos de 4 letras, podríamos obtener palabras más relevantes.

In [16]:
df = Vocabulary[Vocabulary.words.str.len() > 4]
df[0:10]

,words,frequency
16,había,858
26,señor,447
28,hombre,363
35,obispo,286
37,estaba,273
38,sobre,269
39,cuando,265
41,aquel,247
48,madeleine,214
49,tenía,211


In [17]:
len(df)

12435